In [ ]:
import time
import torch
from torch.utils.data import DataLoader, TensorDataset
from torchvision.io import read_video
from torchvision.transforms.functional import resize,normalize
import ultralytics
import depth_pro

ultralytics.checks()
model = ultralytics.YOLO("yolo11x-pose.pt")

In [ ]:
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
rat = a/r
print(r,a,f,rat)

In [ ]:
# Load video to tensor
v_frames = read_video("Homebrew-video/Low-quality/IMG_5347.MP4",output_format="TCHW")[0].to("cuda").half()/255
v_frames = normalize(v_frames,[0.5,0.5,0.5],[0.5,0.5,0.5],inplace=True)
#v_frames = DataLoader(v_frames)
#v_frames = DataLoader(TensorDataset(v_frames),batch_size=v_frames.shape[0])
#v_frames = read_video("Homebrew-video/Low-quality/IMG_5347.MP4")[0].to("cpu")
#print(v_frames)

In [ ]:
# Load depth model and preprocessing transform
depth_model, transform = depth_pro.create_model_and_transforms(device="cuda",precision=torch.half)
depth_model.eval()
print("all good")

In [ ]:
from torchvision.utils import save_image,

for i,v_frame in enumerate(v_frames):
    depth_in = time.time()

    # Run inference.
    prediction = depth_model.infer(v_frame)
    depth = prediction["depth"]  # Depth in [m].
    focallength_px = prediction["focallength_px"]  # Focal length in pixels.
    
    depth_out = time.time()
    
    print(depth_out-depth_in)
    
    #print(depth)
    #save_image(v_frame,f"frame_{i}.png")
    

In [ ]:
depth_in = time.time()
# Run inference.
prediction = depth_model.infer(v_frames)
depth = prediction["depth"]  # Depth in [m].
focallength_px = prediction["focallength_px"]  # Focal length in pixels.

depth_out = time.time()

print(depth_out-depth_in)

In [ ]:
# Run online object detection
results = model.track(source="Homebrew-video/Low-quality/IMG_5347.MP4",classes=[0],stream=True)
start = time.time()
# Loop over YOLO results and call inference on depth pro
for i,r in enumerate(results):
    depth_in = time.time()
    # Transform image
    #d_img = transform(v_frames[i])
    # Run inference.
    prediction = depth_model.infer(v_frames[i])
    depth = prediction["depth"]  # Depth in [m].
    focallength_px = prediction["focallength_px"]  # Focal length in pixels.
    depth_out = time.time()
    print(depth_out-depth_in)
    end = time.time()
    print(end-start)

In [ ]:
#beeg = resize(v_frames,(640,640)) ; print(beeg.shape)
#results = model.predict(source=beeg, save=True)

for frame in v_frames:
    frame = resize(frame,600,max_size=640).unsqueeze(0)/255
    results = model.track(source=frame,save=True,classes=[0])